# Wedding Seating Optimization

This work was conducted by Group G:

- Daniel Caridade (20211588)
- Gonçalo Teles (20211684)
- Gonçalo Peres (20211625)
- Guilherme Godinho (20211552)

# 1. Library Importation

__`Step 1`__ Import the required libraries.

In [1]:
import pandas as pd
import random
import numpy as np
import copy
import math

# 2. Data Integration

__`Step 2`__ Importing the dataset into the file.

In [2]:
from utils.parser import load_relationship_matrix

# Load the data
relationships = load_relationship_matrix()
print(relationships.head())

       1     2     3     4    5    6    7    8    9    10  ...   55   56  57  \
idx                                                        ...                 
1       0  5000     0     0  700  700    0    0    0    0  ...  100  100   0   
2    5000     0   700   700    0    0  300  300  500  500  ...  100  100   0   
3       0   700     0  2000    0    0    0    0  300  300  ...    0    0   0   
4       0   700  2000     0    0    0  900  400  300  300  ...    0    0   0   
5     700     0     0     0    0    0    0    0    0    0  ...    0    0   0   

      58   59   60   61  62  63  64  
idx                                  
1      0  100  100  100   0   0   0  
2    100    0    0    0   0   0   0  
3      0    0    0    0   0   0   0  
4      0    0    0    0   0   0   0  
5      0    0    0    0   0   0   0  

[5 rows x 64 columns]


# 3. Helper functions

__`Step 3`__

In [4]:
from utils.WeddingSeatingHelper import WeddingSeatingHelper # adjust import path accordingly

# Suppose relationships is loaded before, e.g. from your parser:
helper = WeddingSeatingHelper(relationships)

## Hill Climbing testing

In [5]:
from optimizers.HillClimbing import HillClimbingOptimizer

helper = WeddingSeatingHelper(relationships)
hc = HillClimbingOptimizer(helper)

best_fit, best_sol = hc.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 19900
Current fitness: 24400
Current fitness: 28700
Current fitness: 32900
Current fitness: 37000
Current fitness: 40600
Current fitness: 44000
Current fitness: 47300
Current fitness: 50300
Current fitness: 53100
Current fitness: 55800
Current fitness: 58400
Current fitness: 60800
Current fitness: 62700
Current fitness: 64300
Current fitness: 65700
Current fitness: 67700
Current fitness: 68400
Current fitness: 69000
Current fitness: 69600
Current fitness: 69800
Current fitness: 69900
Best fitness: 69900


Testing for several iterations

In [ ]:
results_hc = []

for i in range(30):
    hc = HillClimbingOptimizer(helper, seed=i)  # different seed for each run
    best_fit, best_sol = hc.run()
    results_hc.append(best_fit)
    print(f"Run {i+1}: Best fitness = {best_fit}")

print(f"\nAverage fitness: {sum(results_hc)/len(results_hc):.2f}")
print(f"Best fitness: {max(results_hc)}")
print(f"Worst fitness: {min(results_hc)}")

## Simulated Annealing in classes 

In [7]:
from optimizers.SimulatedAnnealing import SimulatedAnnealingOptimizer

# Run Simulated Annealing
sa = SimulatedAnnealingOptimizer(helper)
best_fit, best_sol = sa.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 16300
Current fitness: 21300
Current fitness: 26300
Current fitness: 30700
Current fitness: 35100
Current fitness: 39200
Current fitness: 42500
Current fitness: 46000
Current fitness: 49000
Current fitness: 51800
Current fitness: 54700
Current fitness: 57400
Current fitness: 59800
Current fitness: 62400
Current fitness: 64500
Current fitness: 66600
Current fitness: 68600
Current fitness: 70600
Current fitness: 71700
Current fitness: 74200
Current fitness: 75700
Current fitness: 76000
Current fitness: 76100
Current fitness: 76400
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Best fitness: 76500


Testing for several iterations

In [ ]:
results_sa = []

# Create a fresh instance each time to reset random seed or internal state if needed
for i in range(30):
    sa = SimulatedAnnealingOptimizer(helper, seed=i) # different seed for each run
    best_fit, best_sol = sa.run()
    results_sa.append(best_fit)
    print(f"Run {i+1}: Best fitness = {best_fit}")

# Summarize results
print(f"\nAverage fitness over 30 runs: {sum(results_sa)/len(results_sa):.2f}")
print(f"Best fitness found: {max(results_sa)}")
print(f"Worst fitness found: {min(results_sa)}")

## Genetic Algorithms

In [ ]:
from optimizers.genetic_algorithm.selection import selection

# Generate a random population
population = [helper.generate_solution() for _ in range(50)]  # pop size = 50

# Try battle selection
selected = selection(population, pop_size=50, helper=helper, selection="battle", elitism=True)

# Try double roulette
selected_roulette = selection(population, pop_size=50, helper=helper, selection="double_roullette")

In [ ]:
from optimizers.genetic_algorithm.mutation import mutation

# Create a sample individual: 8 tables with 8 guests each
sample_individual = [list(range(i*8 + 1, i*8 + 9)) for i in range(8)]

# Apply mutation
mutated = mutation(sample_individual.copy(), swap=True, table_flip=False, relationship_augmenter=False)
display(mutated)

In [ ]:
# Future code for crossover


In [ ]:
from optimizers.genetic_algorithm.genetic_algorithm import GeneticAlgorithm

ga = GeneticAlgorithm(pop_size=50, num_gen=20, selection_method="battle", elitism=True)
best_score, best_solution = ga.optimize()

Testing for several times

In [ ]:
results_ga = []

for i in range(30):
    random.seed(i)
    np.random.seed(i)
    
    ga = GeneticAlgorithm(pop_size=50, num_gen=20, selection_method="battle", elitism=True)
    best_fit, best_sol = ga.optimize(verbose=False)  # disable verbose logging
    
    results_ga.append(best_fit)
    print(f"GA Run {i+1}: Best fitness = {best_fit:.2f}")

# Summarize results
print(f"\nGA Average fitness over 30 runs: {np.mean(results_ga):.2f}")
print(f"GA Best fitness found: {np.max(results_ga):.2f}")
print(f"GA Worst fitness found: {np.min(results_ga):.2f}")